# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import re

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Error loading stopwords: <urlopen error [Errno 61]
[nltk_data]     Connection refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading wordnet: <urlopen error [Errno 61]
[nltk_data]     Connection refused>


In [2]:
# load data from database
engine = create_engine('sqlite:///messages.db')
conn = engine.connect()
df = pd.read_sql('SELECT * FROM messages', con=conn)
df.head()
X = df['message']
Y = df.iloc[:, 4: ]
X.head(10)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
5               Information about the National Palace-
6                       Storm at sacred heart of jesus
7    Please, we need tents and water. We are in Sil...
8      I would like to receive the messages, thank you
9    I am in Croix-des-Bouquets. We have health iss...
Name: message, dtype: object

In [3]:
Y

,0,1,2,3,4,5,6,7,8,9,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    '''Function normalizes case, removes punctuation, stems, lemmatizes and parse a message into separate words.
    '''
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10, max_depth=5, max_features=0.4,\
                                                                  max_sample=0.8)))
])

In [13]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f28b781a9d8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
               oob_score=False, random_state=None

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y)

pipeline.fit(Xtrain, Ytrain)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
Ypred = pipeline.predict(Xtest)

In [16]:
# turn Ypred into a dataframe
Ypred = pd.DataFrame(Ypred)
Ypred.columns = Ytest.columns

#check the confusion matrix for a specific output category.
confusion_matrix(Ytest.iloc[:,0], Ypred.iloc[:,0])

array([[ 610,  949],
       [ 247, 4748]])

In [17]:
r1 = classification_report(Ytest.iloc[:,0], Ypred.iloc[:,0])
r1 = classification_report(Ytest['related'], Ypred['related'])

report = classification_report(Ytest, Ypred, target_names = Ytest.columns)

print(r1)
print(report)

             precision    recall  f1-score   support

          0       0.72      0.39      0.51      1559
          1       0.83      0.95      0.89      4995

avg / total       0.81      0.82      0.80      6554

                        precision    recall  f1-score   support

                     0       0.83      0.95      0.89      4995
                     1       0.85      0.49      0.62      1116
                     2       0.00      0.00      0.00        31
                     3       0.77      0.70      0.73      2703
                     4       0.65      0.11      0.19       490
                     5       0.82      0.12      0.21       341
                     6       0.67      0.05      0.08       177
                     7       0.00      0.00      0.00       106
                     8       0.65      0.05      0.09       227
                     9       0.00      0.00      0.00         0
                    10       0.90      0.41      0.56       388
                

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [19]:
import pickle

pkl_filename = 'pickle_RFmodel.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(pipeline, file)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# Note, parameter search with CV is way too slow
parameters = {
    # determine exact parameter names from pipeline.get_params()
    'vect__min_df': [0.005, 0.01]
    'vect__max_df': [0.25, 0.5, 1.0]
    #'clf__estimator__n_estimators': [100, 200],
    #'clf__estimator__max_features': ['sqrt', 'log2'],
    #'clf__estimator__min_samples_leaf': [10,20]
}

#cv = GridSearchCV(pipeline, param_grid=parameters)
#cv.fit(Xtrain, Ytrain)

In [111]:
# Use cross validation to search for the best parameters turns out to be much too slow. 
# Since we are doing random forest, we can take advantage of the bootstrap process and check the OOB errors.
# Will this be a little bit faster?
pipeline1 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(oob_score=True, n_estimators=50, min_samples_leaf=1)))
])
pipeline1.fit(Xtrain, Ytrain)

est = pipeline1.steps[2][1].estimators_
oob1 = []
for e in est:
    oob1.append(e.oob_score_)
sum(oob1)/len(oob1)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...
            oob_score=True, random_state=None, verbose=0, warm_start=False),
           n_jobs=1))])

In [102]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(oob_score=True, n_estimators=50, min_samples_leaf=10)))
])
pipeline2.fit(Xtrain, Ytrain)

est = pipeline2.steps[2][1].estimators_
oob2 = []
for e in est:
    oob2.append(e.oob_score_)
sum(oob2)/len(oob2)

0.93996999135344084

In [104]:
pipeline3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(oob_score=True, n_estimators=100, min_samples_leaf=5)))
])
pipeline3.fit(Xtrain, Ytrain)


est = pipeline3.steps[2][1].estimators_
oob3 = []
for e in est:
    oob3.append(e.oob_score_)
sum(oob3)/len(oob3)

0.94215706220436379

In [113]:
pipeline4 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(oob_score=True, n_estimators=100, min_samples_leaf=1)))
])
pipeline4.fit(Xtrain, Ytrain)

est = pipeline4.steps[2][1].estimators_
oob4 = []
for e in est:
    oob4.append(e.oob_score_)
sum(oob4)/len(oob4)

0.94773070206669729

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [115]:
Ypredoob = pipeline4.predict(Xtest)

In [116]:
Ypredoob = pd.DataFrame(Ypredoob)
Ypredoob.columns = Ytest.columns
r1oob = classification_report(Ytest['related'], Ypredoob['related'])
reportoob = classification_report(Ytest, Ypredoob, target_names = Ytest.columns)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [118]:
print(r1oob)
print(reportoob)

             precision    recall  f1-score   support

          0       0.70      0.39      0.50      1562
          1       0.83      0.95      0.89      4992

avg / total       0.80      0.81      0.79      6554

                        precision    recall  f1-score   support

               related       0.83      0.95      0.89      4992
               request       0.83      0.50      0.62      1088
                 offer       0.00      0.00      0.00        23
           aid_related       0.76      0.70      0.73      2723
          medical_help       0.65      0.07      0.12       522
      medical_products       0.72      0.11      0.18       312
     search_and_rescue       0.77      0.06      0.10       179
              security       0.00      0.00      0.00       111
              military       0.64      0.08      0.15       220
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.37      0.52       425
                

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [21]:
# Try random forests in XGBoost
pipeline_xgb = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=xgb.XGBClassifier(objective='binary:logistic', colsample_bynode=0.4,\
                                                             learning_rate=1, max_depth=5, num_parallel_tree=10,\
                                                             subsample=0.8, eval_metric='logloss',\
                                                             use_label_encoder=False)))
])
##ATTENTION: how does use_label_encoder affect the result?

In [22]:
pipeline_xgb.fit(Xtrain, Ytrain)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...e,
       use_label_encoder=False, validate_parameters=None, verbosity=None),
           n_jobs=1))])

In [23]:
pkl_filename = 'pickle_XGBRFmodel.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(pipeline_xgb, file)

In [55]:
Ypred = pipeline_xgb.predict(Xtest)

In [57]:
report = classification_report(Ytest, Ypred, target_names = Ytest.columns)
print(report)

                        precision    recall  f1-score   support

               related       0.86      0.93      0.89      5014
               request       0.75      0.60      0.67      1084
                 offer       0.00      0.00      0.00        34
           aid_related       0.76      0.70      0.73      2693
          medical_help       0.59      0.34      0.43       524
      medical_products       0.67      0.39      0.49       340
     search_and_rescue       0.57      0.22      0.32       162
              security       0.15      0.02      0.03       104
              military       0.62      0.41      0.50       205
           child_alone       0.00      0.00      0.00         0
                 water       0.77      0.67      0.71       414
                  food       0.81      0.76      0.78       693
               shelter       0.75      0.60      0.67       606
              clothing       0.76      0.46      0.57       112
                 money       0.52      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [58]:
def display_result(Ypred, Ytest):
    Ypred = pd.DataFrame(Ypred)
    Ypred.columns = Ytest.columns
    report = classification_report(Ytest, Ypredoob, target_names = Ytest.columns)
    print(report)
    return 0

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.